(usage/table)=
# Table
<hr class="sn-grey">
<a 
    class="sphinx-bs badge badge-primary text-white reference external sn-api sn-link-container2"
    href="../autoapi/snowmobile/core/table/index.html" 
    title="API Documentation">
    <span>snowmobile.core.table</span>
</a>

```{admonition} Warning: This pge is incomplete
 :class: error, sn-inherit-overflow
 &nbsp;
```

{class}`snowmobile.Table<snowmobile.core.table.Table>` is a data loading 
solution that implements the {xref}`bulk loading from a local file system` to 
load a {class}`~pandas.DataFrame` into a table; an instance can be instantiated
from defaults with a `table` (name {class}`str`) and a `df` 
({class}`~pandas.DataFrame`).

Exhaustive documentation on its keyword arguments can be found in
[the API docs](../autoapo/snowmobile/core/table/index.html), with additional
detail on the most broadly applicable included below. 

```{admonition} Note
:class: note
Alternate default values for the keyword arguments documented below can be
specified in the `default-table-kwargs` section of [](./snowmobile_toml.md) 
```

<hr class="sn-grey">

## Examples
---

#### *Setup*

```{div} sn-pre-cell
In the following examples, the variable **df** is reserved for the
`df` created with:
```

In [1]:
"""Generates reproducible DataFrame for snowmobile.Table examples"""
import datetime
import pandas as pd
import numpy as np

from typing import List

def rnp(c: List[str], seed: int = 1) -> str:
    """Returns a randomly chosen name within a list, c."""
    np.random.seed(seed)
    return np.random.choice(c)


data = {
    'col1': [rnp(['spam', 'eggs', 'ham']) for _ in range(24)],
    'col2': [rnp(['alpha', 'beta', 'gamma']) for _ in range(24)],
    'col3': [i for i in range(24)],
}

df = pd.DataFrame(data=data)
# print(df.shape)  #> (24, 3)
df.head(4)

# -- complete example; should run 'as is' --

,col1,col2,col3
0,eggs,beta,0
1,eggs,beta,1
2,eggs,beta,2
3,eggs,beta,3


As is the variable **sn** for the `sn` created with:

In [5]:
"""All snowmobile objects start with one of these"""
import snowmobile

sn = snowmobile.connect()

Locating credentials...
(1 of 2) Checking for cached path...
(2 of 2) Cached path found at ../Snowmobile/snowmobile.toml
..connected: snowmobile.Snowmobile(creds='default')


In [6]:
sn.sql.drop('sample_table')

,status
0,SAMPLE_TABLE successfully dropped.


\
*If the state of either needs to be altered in order for a specific context to be reproducible,
it will be explicitly documented and reset at the beginning of the following example.*

### {class}`~pandas.DataFrame` to New Table
<hr class="sn-green-thick">

Loads a {class}`~pandas.DataFrame` into a *sample_table* that hasn't been created yet; falling back
to entirely default [snowmobile.toml](./snowmobile_toml.md) values.

In [7]:
# We can load it like this
t1 = snowmobile.Table(df=df, sn=sn, table='sample_table', as_is=True)

Loading into 'gem7318.SAMPLE_TABLE`..
(1 of 4)
	CREATE OR REPLACE TABLE SAMPLE_TABLE ( ..
(2 of 4)
	create stage SAMPLE_TABLE_stage file_format = snowmobile_default_psv;
(3 of 4)
	put file://C:/Users/GEM7318/Documents/Github/Snowmobile/sample_table.csv @SAMPLE_TABLE_stage
	auto_compress = true
(4 of 4)
	copy into SAMPLE_TABLE from @SAMPLE_TABLE_stage
	on_error = continue
..completed: 24 rows in 2 seconds


Here's some info about `t1` and *sample_table*:

In [8]:
print(t1.loaded)                           #> True
print(sn.sql.exists('sample_table'))       #> True

print(t1.df.shape)                         #> (24,3)
print(sn.sql.count('sample_table'))  #> [24]

True
True
(24, 3)
[24]


&nbsp;

&nbsp;

### {class}`~pandas.DataFrame` to Existing Table (valid for load)
<hr class="sn-green-thick">

```{admonition}

This example picks up where the above example left off.
```

Let's try it again but tell it to yell at us if the table already exists by providing `if_exists='fail'`:

In [9]:
from snowmobile.core.errors import ExistingTableError

try:
    _ = snowmobile.Table(
        df=df,
        sn=sn,
        table='sample_table',
        as_is=True,
        if_exists='fail',
    )
except ExistingTableError as e:
    print(e)

`SAMPLE_TABLE` already exists and if_exists='fail' was provided;
'replace', 'append', or 'truncate' required to continue load process with a pre-existing table.


```{admonition} TODO
:class: error
Finish from here down
```

In [10]:
sn.sql.select('sample_table', n=3)

,col1,col2,col3
0,eggs,beta,0
1,eggs,beta,1
2,eggs,beta,2


In [11]:
sn.sql.count('sample_table')

[24]

### {class}`~pandas.DataFrame` to Existing Table (invalid for load)
<hr class="sn-green-thick">

In [12]:
# print(t1.loaded)

# t12 = snowmobile.Table(sn=sn, table=TESTING_TABLE_NAME, df=df).load()


# Ex. 11 ----------------------------------------------------------------------

from snowmobile.core.errors import ExistingTableError

try:
    snowmobile.Table(
        sn=sn,
        table=TESTING_TABLE_NAME,
        df=df,
        as_is=True,
        if_exists='fail',
    )
except ExistingTableError as e:
    print(e)


# Ex. 2 -----------------------------------------------------------------------

df2 = pd.concat([df, df], axis=1)

t2 = snowmobile.Table(sn=sn, table=TESTING_TABLE_NAME, df=df2)
t2.df.head()

for c1, c2 in zip(df2.columns, t2.df.columns):
    print(f"df2.{c1} t2.df.{c2}")

from snowmobile.core.errors import ColumnMismatchError

try:
    t2.load(if_exists='append')
except ColumnMismatchError as e:
    print(e)


# Ex. 22 ----------------------------------------------------------------------

df22 = pd.concat([df, df], axis=1)


t22 = snowmobile.Table(sn=sn, table=TESTING_TABLE_NAME, df=df22, check_dupes=False)
t2.cols_match  #> False

t22.load(if_exists='replace')

from snowmobile.core.errors import ColumnMismatchError

try:
    t2.load(if_exists='append')
except ColumnMismatchError as e:
    print(e)


# Ex. 3 -----------------------------------------------------------------------

indices = list(range(1, 5))                   # [1, 2, 3, 4]
idx_to_concat = rnp().random.choice(indices)  # 2
dfs = {
    i: df if i != idx_to_concat
    else pd.concat([df, df], axis=1)
    for i in indices
}

for i, df in dfs.items():
    print(f"{i}: {df.shape[0]}x{df.shape[1]}")

from typing import Dict

tables: Dict[int, snowmobile.Table] = {
    i: snowmobile.Table(
        sn=sn,
        table=TESTING_TABLE_NAME,
        df=df,
        as_is=True,
        on_error='c',  # <-- suppresses & stores the exception that would be raised
    )
    for i, df in dfs.items()
}

for i, table in tables.items():
    print(f"({i}) loaded: {table.loaded}")

try:
    raise tables[2].e.get(to_raise=True, last=True)
except ColumnMismatchError as e:
    print(e)

# Ex. 2.1 ---------------------------------------------------------------------


# Ex. 3 -----------------------------------------------------------------------

sn.sql.exists(TESTING_TABLE_NAME)

df3 = df.copy(deep=True).rename(
    columns={
        'col1': 'col1 $#$% 3 )))_____34324'
    }
)

t1 = snowmobile.Table(sn=sn, table=TESTING_TABLE_NAME, df=df3)
t1.df.head()

NameError: name 'TESTING_TABLE_NAME' is not defined

## *as_is=True* vs. *Table.load()*
---

{class}`~snowmobile.Table` has a {meth}`~snowmobile.Table.load()` that enables pre-inspection of the {class}`~snowmobile.Table` object before kicking it off.